# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [111]:
# Correct

import pandas as pd
import numpy as np
import scipy.stats as stats
import re


#print(nhl_df)

def calcTeamName(row):
    teamNameStr = row['NHL']
        
    if teamNameStr == '':
        row['NHL'] = np.NaN
    elif teamNameStr.find(' ') == -1:
        teams = re.findall('[A-Z][^A-Z]*', teamNameStr)
        row['NHL'] = teams
    else:
        row['NHL'] = [teamNameStr]

    return row

def calcFullTeamName(row):
    area = row['Area']
    teamName = row['NHL']

    if area == 'New York City':
        area = 'New York'
    elif area =='Las Vegas':
        area = 'Vegas'
    elif area =='Tampa Bay Area':
        area = 'Tampa Bay'

    if teamName == 'Devils':
        area = 'New Jersey'
    elif teamName == 'Ducks':
        area = 'Anaheim'
    elif teamName == 'Sharks':
        area = 'San Jose'
    elif teamName == 'Stars':
        area = 'Dallas'
    elif teamName == 'Capitals':
        area = 'Washington'
    elif teamName == 'Wild':
        area = 'Minnesota'
    elif teamName == 'Avalanche':
        area = 'Colorado'
    elif teamName == 'Panthers':
        area = 'Florida'
    elif teamName == 'Coyotes':
        area = 'Arizona'
    elif teamName == 'Hurricanes':
        area = 'Carolina'

    fullTeamName = area + ' ' + teamName
    row['FullTeamName'] = fullTeamName
    return row

def calcWinLoss(row):
    wins = int(row['W'])
    losses = int(row['L'])
    winlossratio = wins / (wins + losses)
    row['Win Loss Ratio'] = winlossratio
    return row


def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
  
    myCities = cities.copy()
    #df[df['P_NUMVRC'] > 0]
    myNHL = nhl_df[nhl_df['year'] == 2018].copy()

    myCities.columns=['Area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    myCities = myCities[['Area', 'Population', 'NHL']]
    myCities['Population'] = myCities['Population'].astype(int)
    
    myCities = myCities.replace(to_replace=r'^—$', value=np.NaN, regex=True)
    myCities = myCities.replace(to_replace=r'(\[note [\d]+\])', value="", regex=True)
    myCities = myCities.dropna()
   
    myCities = myCities.apply(calcTeamName, axis='columns')
    myCities = myCities.dropna()
    myCities = myCities.explode('NHL')

    myCities = myCities.apply(calcFullTeamName, axis='columns')

    #print(myCities)

    myNHL = myNHL[['team', 'W', 'L', 'year']]
    myNHL = myNHL.replace(to_replace=r'([A-Z][a-z]* Division)', value=np.NaN, regex=True)
    myNHL = myNHL.replace(to_replace=r'\*', value='', regex=True)
    myNHL = myNHL.dropna()
    
    myNHL = myNHL.apply(calcWinLoss, axis='columns')

    df = pd.merge(myCities, myNHL, how='outer', left_on='FullTeamName', right_on='team')
    
    df = df[['Area', 'Population', 'team', 'Win Loss Ratio']]
    #print(df)
    df2 = df.groupby("Area", sort=False).agg({"Win Loss Ratio": (np.mean), "Population": (np.mean)})
    #print(df2)
    #print(len(df2))

    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = df2['Win Loss Ratio'].tolist()
    population_by_region = myCities[['Area', 'Population']].drop_duplicates()['Population'].tolist()

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

nhl_correlation()

Area  Population             NHL           FullTeamName
0            New York City    20153634         Rangers       New York Rangers
0            New York City    20153634       Islanders     New York Islanders
0            New York City    20153634          Devils      New Jersey Devils
1              Los Angeles    13310447           Kings      Los Angeles Kings
1              Los Angeles    13310447           Ducks          Anaheim Ducks
2   San Francisco Bay Area     6657982          Sharks        San Jose Sharks
3                  Chicago     9512999      Blackhawks     Chicago Blackhawks
4        Dallas–Fort Worth     7233323           Stars           Dallas Stars
5         Washington, D.C.     6131977        Capitals    Washington Capitals
6             Philadelphia     6070500          Flyers    Philadelphia Flyers
7                   Boston     4794447          Bruins          Boston Bruins
8   Minneapolis–Saint Paul     3551036            Wild         Minnesota Wild
9       

0.012486162921209912

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [155]:
# Correct
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def calcTeamName(row):
    teamNameStr = row['NBA']
    if teamNameStr == '':
        row['NBA'] = np.NaN
    elif teamNameStr == '76ers':
        row['NBA'] == [teamNameStr]
    elif teamNameStr.find(' ') == -1:
        teams = re.findall('[A-Z][^A-Z]*', teamNameStr)
        row['NBA'] = teams
    else:
        row['NBA'] = [teamNameStr]

    return row


def calcFullTeamName(row):
    area = row['Area']
    teamName = row['NBA']

    if area == 'New York City':
        area = 'New York'
    elif area == 'San Francisco Bay Area':
        area = 'Golden State'
    elif area == 'Dallas–Fort Worth':
        area = 'Dallas'
    elif area == 'Washington, D.C.':
        area = 'Washington'
    elif area == 'Minneapolis–Saint Paul':
        area = 'Minnesota'
    elif area == 'Miami–Fort Lauderdale':
        area = 'Miami'
    elif area == 'Indianapolis':
        area = 'Indiana'
    elif area == 'Salt Lake City':
        area = 'Utah'

    if(teamName == 'Nets'):
        area = 'Brooklyn'

    fullTeamName = area + ' ' + teamName
    row['FullTeamName'] = fullTeamName
    return row

def calcWinLoss(row):
    wins = int(row['W'])
    losses = int(row['L'])
    winlossratio = wins / (wins + losses)
    row['Win Loss Ratio'] = winlossratio
    return row

def nba_correlation():

    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    myNBA = nba_df[nba_df['year'] == 2018].copy()
    
    myCities = cities.copy()

    myCities.columns=['Area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    myCities = myCities[['Area', 'Population', 'NBA']]
    myCities['Population'] = myCities['Population'].astype(int)
    
    myCities = myCities.replace(to_replace=r'^—$', value=np.NaN, regex=True)
    myCities = myCities.replace(to_replace=r'(\[note [\d]+\])', value="", regex=True)
    myCities = myCities.dropna()
    myCities = myCities.apply(calcTeamName, axis='columns')
    myCities = myCities.dropna()
    myCities = myCities.explode('NBA')

    myCities = myCities.apply(calcFullTeamName, axis='columns')
    #print(myCities)

    myNBA = myNBA[['team', 'W', 'L', 'year']]
    myNBA = myNBA.replace(to_replace=r'\*', value='', regex=True)
    myNBA['team'] = myNBA['team'].replace(to_replace=r'(\s*\([\d]+\))', value='', regex=True)
    myNBA = myNBA.dropna()
    myNBA = myNBA.apply(calcWinLoss, axis='columns')
    #print(myNBA)
    df = pd.merge(myCities, myNBA, how='outer', left_on='FullTeamName', right_on='team')
    df = df[['Area', 'Population', 'team', 'Win Loss Ratio']]
    #print(df)
    df2 = df.groupby("Area", sort=False).agg({"Win Loss Ratio": (np.mean), "Population": (np.mean)})
    #print(df2)
    
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = df2['Win Loss Ratio'].tolist()
    population_by_region = myCities[['Area', 'Population']].drop_duplicates()['Population'].tolist()

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

Win Loss Ratio  Population
Area                                              
New York City                 0.347561    20153634
Los Angeles                   0.469512    13310447
San Francisco Bay Area        0.707317     6657982
Chicago                       0.329268     9512999
Dallas–Fort Worth             0.292683     7233323
Washington, D.C.              0.524390     6131977
Philadelphia                  0.634146     6070500
Boston                        0.670732     4794447
Minneapolis–Saint Paul        0.573171     3551036
Denver                        0.560976     2853077
Miami–Fort Lauderdale         0.536585     6066387
Phoenix                       0.256098     4661537
Detroit                       0.475610     4297617
Toronto                       0.719512     5928040
Houston                       0.792683     6772470
Atlanta                       0.292683     5789700
Cleveland                     0.609756     2055612
Charlotte                     0.439024     2474314
Indi

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [180]:
# Correct
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def calcTeamName(row):
    teamNameStr = row['MLB']
    
    if teamNameStr == '':
        row['MLB'] = np.NaN
    elif teamNameStr == 'CubsWhite Sox':
        row['MLB'] = ['Cubs', 'White Sox']
        print('here')
    elif teamNameStr.find(' ') == -1:
        teams = re.findall('[A-Z][^A-Z]*', teamNameStr)
        row['MLB'] = teams
    else:
        row['MLB'] = [teamNameStr]

    return row


def calcFullTeamName(row):
    area = row['Area']
    teamName = row['MLB']

    if area == 'New York City':
        area = 'New York'
    
    if teamName == 'Athletics':
        area = 'Oakland'
    elif teamName == 'Giants':
        area = 'San Francisco'
    elif teamName == 'Rangers':
        area = 'Texas'
    elif teamName == 'Nationals':
        area = 'Washington'
    elif teamName == 'Twins':
        area = 'Minnesota'
    elif teamName == 'Rockies':
        area = 'Colorado'
    elif teamName == 'Marlins':
        area = 'Miami'
    elif teamName == 'Diamondbacks':
        area = 'Arizona'
    elif teamName == 'Rays':
        area = 'Tampa Bay'
   
    fullTeamName = area + ' ' + teamName
    row['FullTeamName'] = fullTeamName
    return row

def calcWinLoss(row):
    wins = int(row['W'])
    losses = int(row['L'])
    winlossratio = wins / (wins + losses)
    row['Win Loss Ratio'] = winlossratio
    return row

def mlb_correlation():

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    myMLB = mlb_df[mlb_df['year'] == 2018].copy()
    
    myCities = cities.copy()

    myCities.columns=['Area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    myCities = myCities[['Area', 'Population', 'MLB']]
    myCities['Population'] = myCities['Population'].astype(int)
    
    myCities = myCities.replace(to_replace=r'^—$', value=np.NaN, regex=True)
    myCities = myCities.replace(to_replace=r'(\[note [\d]+\])', value="", regex=True)
    myCities = myCities.dropna()
    
    myCities = myCities.apply(calcTeamName, axis='columns')
    myCities = myCities.dropna()
    myCities = myCities.explode('MLB')
    myCities = myCities.apply(calcFullTeamName, axis='columns')
    #print(myCities)

    myMLB = myMLB[['team', 'W', 'L', 'year']]
    #print(myMLB)
    #myMLB = myNBA.replace(to_replace=r'\*', value='', regex=True)
    #myMLB['team'] = myMLB['team'].replace(to_replace=r'(\s*\([\d]+\))', value='', regex=True)
    myMLB = myMLB.dropna()
    myMLB = myMLB.apply(calcWinLoss, axis='columns')
    #print(myNBA)
    df = pd.merge(myCities, myMLB, how='outer', left_on='FullTeamName', right_on='team')
    df = df[['Area', 'Population', 'team', 'Win Loss Ratio']]
    #print(df)
    df2 = df.groupby("Area", sort=False).agg({"Win Loss Ratio": (np.mean), "Population": (np.mean)})
    #print(df2)
    
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = df2['Win Loss Ratio'].tolist()
    population_by_region = myCities[['Area', 'Population']].drop_duplicates()['Population'].tolist()

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q2: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

here
                        Win Loss Ratio  Population
Area                                              
New York City                 0.546296    20153634
Los Angeles                   0.529122    13310447
San Francisco Bay Area        0.524691     6657982
Chicago                       0.482769     9512999
Dallas–Fort Worth             0.413580     7233323
Washington, D.C.              0.506173     6131977
Philadelphia                  0.493827     6070500
Boston                        0.666667     4794447
Minneapolis–Saint Paul        0.481481     3551036
Denver                        0.558282     2853077
Miami–Fort Lauderdale         0.391304     6066387
Phoenix                       0.506173     4661537
Detroit                       0.395062     4297617
Toronto                       0.450617     5928040
Houston                       0.635802     6772470
Atlanta                       0.555556     5789700
Tampa Bay Area                0.555556     3032171
Pittsburgh                

0.1502769830266931

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [223]:
# Correct
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def calcTeamName(row):
    teamNameStr = row['NFL']
    
    if teamNameStr == '':
        row['NFL'] = np.NaN
  #  elif teamNameStr == 'CubsWhite Sox':
  #      row['NFL'] = ['Cubs', 'White Sox']
    elif teamNameStr.find(' ') == -1:
        teams = re.findall('[A-Z][^A-Z]*', teamNameStr)
        row['NFL'] = teams
    else:
        row['NFL'] = [teamNameStr]

    return row


def calcFullTeamName(row):
    area = row['Area']
    teamName = row['NFL']

    if area == 'New York City':
        area = 'New York'
    
    if teamName == 'Raiders':
        area = 'Oakland'
    elif teamName == 'Cowboys':
        area = 'Dallas'
    elif teamName == 'Redskins':
        area = 'Washington'
    elif teamName == 'Patriots':
        area = 'New England'
    elif teamName == 'Vikings':
        area = 'Minnesota'
    elif teamName == 'Dolphins':
        area = 'Miami'
    elif teamName == 'Cardinals':
        area = 'Arizona'
    elif teamName == 'Buccaneers':
        area = 'Tampa Bay'
    elif teamName == 'Panthers':
        area = 'Carolina'
    elif teamName == 'Titans':
        area = 'Tennessee'
   
    fullTeamName = area + ' ' + teamName
    row['FullTeamName'] = fullTeamName
    return row

def calcWinLoss(row):
    wins = int(row['W'])
    losses = int(row['L'])
    winlossratio = wins / (wins + losses)
    row['Win Loss Ratio'] = winlossratio
    return row

def nfl_correlation():

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    myNFL = nfl_df[nfl_df['year'] == 2018].copy()
    
    myCities = cities.copy()

    myCities.columns=['Area', 'Population', 'NFL', 'MLB', 'NBA', 'NHL']
    myCities = myCities[['Area', 'Population', 'NFL']]
    myCities['Population'] = myCities['Population'].astype(int)
       
   
    myCities = myCities.replace(to_replace=r'(\[note [\d]+\])', value="", regex=True)
    myCities['NFL'] = myCities['NFL'].str.strip()
    myCities = myCities.replace(to_replace=r'^—$', value=np.NaN, regex=True)
    myCities = myCities.replace(to_replace=r'', value=np.NaN, regex=True)
    myCities = myCities.dropna()
    myCities = myCities.apply(calcTeamName, axis='columns')
    myCities = myCities.dropna()
    myCities = myCities.explode('NFL')
    myCities = myCities.apply(calcFullTeamName, axis='columns')
    #print(myCities)
    
    myNFL = myNFL[['team', 'W', 'L', 'year']]
    #print(myMLB)
    #myMLB = myNBA.replace(to_replace=r'\*', value='', regex=True)
    myNFL['team'] = myNFL['team'].replace(to_replace=r'(\*)$', value='', regex=True)
    myNFL['team'] = myNFL['team'].replace(to_replace=r'(\+)$', value='', regex=True)
    myNFL['team'] = myNFL['team'].replace(to_replace=r'(AFC [\w]*)$', value=np.NaN, regex=True)
    myNFL['team'] = myNFL['team'].replace(to_replace=r'(NFC [\w]*)$', value=np.NaN, regex=True)
    myNFL = myNFL.dropna()
    
    myNFL = myNFL.apply(calcWinLoss, axis='columns')
    
    
    df = pd.merge(myCities, myNFL, how='outer', left_on='FullTeamName', right_on='team')
    df = df[['Area', 'Population', 'team', 'Win Loss Ratio']]
    #print(df)
    df2 = df.groupby("Area", sort=False).agg({"Win Loss Ratio": (np.mean), "Population": (np.mean)})
    #print(df2)
    
    #population_by_region = [] # pass in metropolitan area population from cities
    #win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = df2['Win Loss Ratio'].tolist()
    population_by_region = myCities[['Area', 'Population']].drop_duplicates()['Population'].tolist()

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values